In [1]:
#include <sstream>
#include <vector>
#include <string>
#include <cmath>
#include <math.h>
#include <cstdlib>
#include <map>
#include "TH1.h"
#include "TH2.h"
#include "TProfile.h"
#include "TCanvas.h"
#include "TGraph.h"
#include "TFile.h"
#include "TString.h"
#include "TStyle.h"

using namespace std;

In [2]:
// Function to fing the edge point by interpolation
double  interpolateTimewalk( TH1D* htw, int Npointsperbin, double &crossingphaseup, double &crossingphasedown){

    TString name  = ""; name+=htw->GetName(); name+=+"_int";
    TString title = name+";interpolated phase; interpolated BXID";

    int npoints = Npointsperbin;
    int nbins   = htw->GetNbinsX();
    TH1D interpoltw(name,"; BXID + phase", npoints*nbins, htw->GetBinLowEdge(1), htw->GetBinLowEdge(nbins+1));
    double precision  = 10;
    double maxpoint   = htw->GetMaximum();
    double edgepoint  = maxpoint*0.30; //  rise and fall edge at 30% of max. 
    double lastintery = 0;
    crossingphaseup   = crossingphaseup = 0;

    for (int interpoint = 0; interpoint<Npointsperbin*htw->GetNbinsX(); interpoint++){
        double interx   = interpoltw.GetBinCenter(interpoint+1);
        double intery   = htw->Interpolate(interx);
        double delta    = (intery-lastintery);
        if(fabs(intery-edgepoint)< precision){
            if(delta>0) crossingphaseup   = interx;
            if(delta<0) crossingphasedown = interx;
        }
        if (fabs(delta) < precision/100.) interpoint+=10;
            lastintery = intery;
    }
    //cout<< " crossingphaseup = " << crossingphaseup << "  crossingphasedown "  << crossingphasedown << endl;
    //cout << "edgepoint = " << edgepoint << endl;
    return 0.5*(crossingphaseup + crossingphasedown);
}

This code performs the analysis demonstrated in Figures 7 and 8 do paper. The output is a .root file were the histograms are stored. This notebook will not plot any of them, as the number of histograms can be too big.

It swipes through the 256 x 256 pixel matrix, but to reduce the size of the set, this version is jumping every 16 rows and 16 columns at

`for(int irow=0; irow<256; irow+=16){    
    for(int icol=0; icol<256; icol+=16){`
    
To increase or decrease the number of pixels analyzed, just change the argument of this loop. But be aware that analysing the entire pixel matrix can take a lot of RAM memory and the notebook can crash.


In [3]:
char myfname[100];
char myhname[100];
char title_up[5000];
char title_low[5000];
char title_avg[5000];

// Input file
TFile *inputfile= TFile::Open("../data/outputfile_negative.root");

// Output file
sprintf(myfname,"../results/htwInterpPix.root");
TFile *outfile = new TFile(myfname, "RECREATE");

cout << "BE PATIENT!! Depending on the number of pixels this might take hours!!" << endl;

// It can be done for all 256 row and 256 columns of the matrix, just change the loop to irow++/icol++. 
// Be aware that analyse the entire pixel matrix can take a lot of RAM memory and the notebook can crash

for(int irow=0; irow<256; irow+=16){     // sweep through rows of the pixel array -> CHANGE TO ANALYSE MORE/LESS PIXELS
    for(int icol=0; icol<256; icol+=16){ // sweep through columns of the pixel array -> CHANGE TO ANALYSE MORE/LESS PIXELS

        // Declaring new histograms
        TH1F *hlowEdge=NULL, *haverageEdge=NULL, *hhighEdge=NULL;
        TGraph *tgr=NULL;
        TH2F *htw=NULL;

        // Get histograms from input file
        sprintf(myhname,"htwAllperpix_r%d_c%d", irow, icol);
        htw =  (TH2F*) inputfile->Get(myhname);

        // Check if histograms from input file exist
        if (htw==NULL){
            //cout << " Histogram " << myhname << " not found. Trying next histogram... " << endl;
            continue;
        }
        else{
            // Titles and axes of the histograms
            sprintf(title_avg,"Crossing phase average at 30 percent - Pixel(%d,%d); #Delta VTP; bxid+(1.0/16)*phase", irow, icol);
            sprintf(title_up, "Crossing phase up at 30 percent - Pixel(%d,%d); #Delta VTP; bxid+(1.0/16)*phase", irow, icol);
            sprintf(title_low,"Crossing phase low at 30 percent - Pixel(%d,%d); #Delta VTP; bxid+(1.0/16)*phase", irow, icol);

            
            int nbins = htw->GetNbinsX();  
            
            // Lower edge histogram
            TString myname =""; myname+=htw->GetName(); myname+="_lowEdge"; 
            hlowEdge = new TH1F(myname.Data(), title_low,
                                htw->GetNbinsX(),
                                htw->GetXaxis()->GetBinLowEdge(1),
                                htw->GetXaxis()->GetBinLowEdge(nbins+1));

            // Middle point historram
            myname =""; myname+=htw->GetName(); myname+="_averageEdge";
            haverageEdge= new TH1F(myname.Data(), title_avg, 
                                   htw->GetNbinsX(), 
                                   htw->GetXaxis()->GetBinLowEdge(1), 
                                   htw->GetXaxis()->GetBinLowEdge(nbins+1));

            // Upper edge histogram
            myname =""; myname+=htw->GetName(); myname+="_highEdge";
            hhighEdge = new TH1F(myname.Data(), title_up, 
                                 htw->GetNbinsX(), 
                                 htw->GetXaxis()->GetBinLowEdge(1), 
                                 htw->GetXaxis()->GetBinLowEdge(nbins+1));

            vector<double> averages;
            vector<double>  mydeltas;
            
            float percentage = 0.3;
            for (int idelta = 1; idelta<513; idelta++){  //dVTP voltages
                //cout << "DeltaVTP-1 = " << idelta << endl << endl;

                TString myname2 =""; myname2 += htw->GetName(); myname2 +="_";myname2 +=idelta;

                //cout << " Analysing histogram " << myname2.Data() << endl;

                // Project input histogram on Y axis (temporary, it is erased)
                TH1D * tempproj= (TH1D*) htw->ProjectionY(myname2.Data(), idelta, idelta);  
                double cpup = 0, cpdown = 0;
                if (irow==0 && icol==0){
                    tempproj->Draw();
                }
                
                // Use an interpolation to find the edge points of the histogram
                interpolateTimewalk(tempproj, 200, cpup, cpdown);
                

                // Fill new histograms
                if( cpup >0.5 && cpdown >0.5){
                    averages.push_back((cpup+cpdown)*0.5);    // Store central value between the rising and falling edges in the "step" histogram 
                    mydeltas.push_back(idelta);               // Store dVTP voltages
                    hlowEdge->SetBinContent(idelta, cpup);    // Fill lower edge histogram
                    hhighEdge->SetBinContent(idelta, cpdown); // Fill middle edge histogram
                    haverageEdge->SetBinContent(idelta,(cpup+cpdown)*0.5); // Fill average point histogram
                }
                
                delete tempproj;

            } //idelta loop
            
            double *myaverages_array = &averages[0]; // Y
            double *mydeltas_array = &mydeltas[0];   // X
            int  Nvals = averages.size();

            // Graph with average curve
            tgr = new TGraph(Nvals,  mydeltas_array, myaverages_array);
            TString finalname = "finalplot_r";
            TString myfinalplot=""; myfinalplot+=finalname; myfinalplot+=Form("%d",(irow));
            myfinalplot+="_c"; myfinalplot+=Form("%d",(icol));
            // cout << endl << " myfinalplot.Data  = " << myfinalplot.Data() << endl;
            if (tgr == NULL){cout << " ???? " << endl; return -2000;}
            tgr->SetName(myfinalplot.Data());
            tgr->SetTitle(title_avg);
            tgr->Draw("ALP");
            
            // Draw egdes/average points together in one histogram
            gStyle->SetOptStat(0);
            hhighEdge->Draw("histp");
            hhighEdge->SetMarkerColor(1);
            hhighEdge->SetMarkerSize(0.7);
            hhighEdge->SetMarkerStyle(20);

            haverageEdge->Draw("histpsame");
            haverageEdge->SetMarkerColor(2);
            haverageEdge->SetMarkerSize(0.7);
            haverageEdge->SetMarkerStyle(20);

            hlowEdge->Draw("histpsame");
            hlowEdge->SetMarkerColor(1);
            hlowEdge->SetMarkerSize(0.7);
            hlowEdge->SetMarkerStyle(20);

            tgr->Write();
            haverageEdge->Write();
            hhighEdge->Write();
            hlowEdge->Write();
            htw->Write();
        }
    }
}

cout << endl << " All indicated pixels analysed " << endl;
inputfile->Close();
outfile->Close();

BE PATIENT!! Depending on the number of pixels this might take hours!!


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
